In [1]:
# Force GeoPandas to use Shapely instead of PyGEOS
# In a future release, GeoPandas will switch to using Shapely by default.
import os
os.environ['USE_PYGEOS'] = '0'

import datacube
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datacube.utils.geometry import Geometry, CRS
from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.dask import create_local_dask_cluster

In [2]:

create_local_dask_cluster()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 1
Total threads: 12,Total memory: 12.15 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41557,Workers: 1
Dashboard: /proxy/8787/status,Total threads: 12
Started: Just now,Total memory: 12.15 GiB
Comm: tcp://127.0.0.1:37197,Total threads: 12
Dashboard: /proxy/35623/status,Memory: 12.15 GiB
Nanny: tcp://127.0.0.1:43863,


In [3]:
# Select a country, for the example we will use Kenya, a complete list of countries is available below.
country = "GUJARAT"

# Set the range of dates for the climatology, this will be the reference period (m) for the anomaly calculation.
# Standard practice is to use a 30 year period, so we've used 1981 to 2011 in this example.
time_m = ('2022', '2024')

# time period for monthly anomaly (x)
time_x = ('2022', '2024')

# CHIRPS has a spatial resolution of ~5x5 km
resolution = (-5000, 5000)

#size of dask chunks
dask_chunks = dict(x=500,y=500)

In [4]:
dc = datacube.Datacube(app='rainfall_anomaly')

In [5]:
indian_countries = gpd.read_file('/home/adithya/Downloads/Tropical_Tree_Cover/Tropical_Tree_Cover.shp')
indian_countries.explore()

In [6]:
print(np.unique(indian_countries.STATE))

AttributeError: 'GeoDataFrame' object has no attribute 'STATE'

In [ ]:
idx = indian_countries[indian_countries['STATE'] == country].index[0]
geom = Geometry(geom=indian_countries.iloc[idx].geometry, crs=indian_countries.crs)

In [ ]:
dc.list_products().loc[dc.list_products()['name'].str.contains('chirps')]

In [ ]:
query = {'geopolygon': geom,
         'time': time_m,
         'output_crs': 'epsg:4326',
         'resolution': resolution,
         'measurements': ['rainfall'],
         'dask_chunks':dask_chunks
        }

ds_rf = dc.load(product='rainfall_chirps_monthly', **query)
print(ds_rf)